# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。
    
    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。
    
    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。
    
    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

In [1]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9, max_tokens=500)

In [2]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [4]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.invoke({
    'product': "性能卓越的GPU"
    }))

{'product': '性能卓越的GPU', 'text': '\n\n1. 光速创力图形芯片有限公司（Lightspeed Creative GPU Co., Ltd.）\n2. 弘图科技有限公司（Grandview Technology Co., Ltd.）\n3. 不止图形处理器有限公司（Beyond Graphics Processor Co., Ltd.）\n4. 极显科技有限责任公司（Extreme Display Technology Ltd.）\n5. 巨像计算机图形有限公司（Gigapixel Computer Graphics Co., Ltd.）\n6. 艺创视界科技有限公司（Artistic Vision Technology Co., Ltd.）\n7. 超纵云图图形解决方案有限公司（Supratemporal Cloud Graphics Solutions Co., Ltd.）\n8. 龙腾芯动科技有限公司（Dragonfly Semiconductor Technology Co., Ltd.）\n9. 游戏引擎加速器有限公司（Game Engine Accelerator Co., Ltd.）\n10. 元耀极致图像有限公司（Elemental Ultimate Imaging Co., Ltd.）'}


In [5]:
chain.verbose = True

In [6]:
chain.verbose

True

In [7]:
print(chain.invoke({
    'product': "性能卓越的GPU"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.

1. 飓岚科技 (Typhoon Technology Co., Ltd.)
2. 弦月智慧 (String Moon Intelligence Co., Ltd.)
3. 极光映像 (Aurora Imaging Co., Ltd.)
4. 神威计算 (Godspeed Computing Co., Ltd.)
5. 艾福特创新 (Epsilon Innovations Co., Ltd.)
6. 灵动芯片 (Dynamic Chip Co., Ltd.)
7. 爆裂视界 (Explosion Vision Co., Ltd.)
8. 超视距科技 (Supersight Technology Co., Ltd.)
9. 驰骋高性能 (Gallop High Performance Co., Ltd.)
10. 异能动力 (Supernormal Power Co., Ltd.)


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](../images/simple_sequential_chain_0.png)

In [8]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。

llm = OpenAI(temperature=0.7, max_tokens=1000)

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [9]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](../images/simple_sequential_chain_1.png)

In [10]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [11]:
review = overall_chain.run("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...


《三体人不是无法战胜的》讲述的是一个关于勇气和希望的故事。在一个遥远的星球上，生活着一群被称为三体人的外星种族。这些生物拥有强大的力量和智慧，一直以来都被认为是无法战胜的存在。

然而，当一群来自地球的勇敢的探险家闯入了他们的领地，他们意外地发现了三体人并不是无敌的。在经历了一系列的挑战和考验后，探险家们发现三体人并不是像外界所传言的那样强大。相反，他们也有着自己的弱点和矛盾。

随着探险家们与三体人的交流和互动，他们渐渐发现了三体人背后的真相和内心的渴望。最终，他们和三体人一起面对困难，共同克服障碍，最终实现了和平与和谐的共存。

《三体人不是无法战胜的》通过展现外星文明的多样性和人类的勇气与智慧，向观众传递了积极的信息：勇敢面对挑战，就能够战胜一切困难，实现和平与共存的梦想。
 

《三体人不是无法战胜的》是一部充满勇气和希望的戏剧作品。通过展现外星文明和人类之间的交流和互动，该剧向观众传递了积极的信息：勇敢面对挑战，就能够战胜一切困难，实现和平与共存的梦想。

该剧以一个遥远的星球上生活着的外星种族——三体人为背景，展现了他们的强大力量和智慧，以及被外界认为是无法战胜的存在。然而，当一群来自地球的勇敢的探险家闯入他们的领地，他们意外发现三体人并不是无敌的。通过一系列的挑战和考验，探险家们发现了三体人的弱点和矛盾，最终实现了和平与和谐的共存。

该剧不仅展现了外星文明的多样性，也通过人类的勇气和智慧向观众传递了积极的信息。在探险家们和三体人的交流和互动中，观众能够感受到勇气和希望的力量。这也让人们意识到，在面对困难和挑战时，勇气和智慧是战胜一切困难的关键。

总的来说，《三体人不是无法战胜的》是一部充满希望和启发的戏剧作品。它不仅展现了外星文明的多样性，也向观众传递了积极的信息：勇敢面对挑战，就能够实现和平与共存的梦想。这部作品值得观众一看。

> Finished chain.


In [12]:
review = overall_chain.invoke("星球大战第九季")



> Entering new SimpleSequentialChain chain...


《星球大战第九季》是一部充满惊险和战斗的科幻戏剧。在这一季中，银河系再次陷入了混乱和冲突之中。经历了前八季的战争和挑战，终于来到了决战的时刻。军事力量和政治势力之间的斗争达到了顶峰，而普通人民也受到了战争的残酷影响。在这个星球大战的最终章中，我们将目睹双方势力之间激烈的战斗，以及来自不同星球和种族的英雄们的顽强抗争。同时，也将有一些意想不到的联盟和背叛发生，为战争增添了更多的变数。最终，谁能够胜出，谁又会葬身战场？《星球大战第九季》将带领观众进入一个充满惊喜和震撼的终极战争世界。


《星球大战第九季》是一部令人兴奋的科幻戏剧，它将观众带入一个充满惊险和战斗的银河系。这部剧集不仅延续了前八季的精彩故事，更将在最终章中展现出更为激烈和激动人心的战争场面。

剧集中，银河系再次陷入混乱和冲突之中，军事力量和政治势力之间的斗争达到了顶峰。同时，普通人民也遭受到战争的残酷影响，这使得剧情更具真实感和触动人心。

在这个终极战争的世界中，我们将目睹来自不同星球和种族的英雄们的顽强抗争，以及一些意想不到的联盟和背叛。这些变数为剧情增添了更多的悬念和紧张感。

导演和演员们在《星球大战第九季》中都表现出色，他们将观众带入一个充满惊喜和震撼的战争世界。每一场战斗都令人血脉贲张，每一位角色都有着鲜明的个性和动人的情感。特效和场景设计也十分出色，为剧集增添了更多视觉上的震撼。

总的来说，《星球大战第九季》是一部令人热血沸腾的戏剧，它将为观众带来一场充满惊喜和震撼的终极战争。无论您是《星球大战》的粉丝还是科幻戏剧的爱好者，这部剧集都值得一看。

> Finished chain.


### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](../images/sequential_chain_0.png)

In [13]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
llm = OpenAI(temperature=.7, max_tokens=1000)
template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [14]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [15]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [17]:
m_overall_chain.invoke({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：

在二十一世纪的新中国，人类社会已经进入了一个繁荣昌盛的时代。然而，随之而来的是一场危机，一群来自外太空的神秘生物——三体人，突然出现在地球上。他们拥有超强的科技实力，令人类束手无策。

面对三体人的入侵，人类陷入了恐慌和绝望之中。但是，在这场看似不可能的战争中，一群勇敢的中国人挺身而出，他们不仅拥有强大的武装力量，更重要的是他们拥有坚定的信念和不屈的精神。

在与三体人的殊死搏斗中，人类逐渐发现，三体人并非无法战胜。他们也有自己的弱点，只是人类需要更深入的了解才能发现。在这场充满挑战的战争中，人类和三体人展开了一场智力和意志的较量。

在最后的决战中，人类和三体人都付出了巨大的牺牲，但最终，人类取得了胜利。通过这场战争，人类也学会了尊重和理解不同的文明，以及珍惜和平与和谐。

《三体人不是无法战胜的》是一部关于科技、信念和人性的戏剧，它向观众呈现了一个未来的世界，也提出了对人类文明的深刻思考。

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '\n在二十一世纪的新中国，人类社会已经进入了一个繁荣昌盛的时代。然而，随之而来的是一场危机，一群来自外太空的神秘生物——三体人，突然出现在地球上。他们拥有超强的科技实力，令人类束手无策。\n\n面对三体人的入侵，人类陷入了恐慌和绝望之中。但是，在这场看似不可能的战争中，一群勇敢的中国人挺身而出，他们不仅拥有强大的武装力量，更重要的是他们拥有坚定的信念和不屈的精神。\n\n在与三体人的殊死搏斗中，人类逐渐发现，三体人并非无法战胜。他们也有自己的弱点，只是人类需要更深入的了解才能发现。在这场充满挑战的战争中，人类和三体人展开了一场智力和意志的较量。\n\n在最后的决战中，人类和三体人都付出了巨大的牺牲，但最终，人类取得了胜利。通过这场战争，人类也学会了尊重和理解不同的文明，以及珍惜和平与和谐。\n\n《三体人不是无法战胜的》是一部关于科技、信念和人性的戏剧，它向观众呈现了一个未来的世界，也提出了对人类文明的深刻思考。',
 'review': '\n\n《三体人不是无法战胜的》是一部令人震撼的戏剧作品，它以未来的中国为背景，讲述了人类与外星种族的殊死搏斗。剧中展现了人类在面对强大的敌人时的勇气、信念和不屈精神，同时也提出了对科技发展、文明冲突和和平共处的深刻思考。\n\n该剧充满了紧张刺激的战斗场面，同时也融入了对于人类情感和内心挣扎的描写。观众可以看到，尽管面对着超强的科技实力，人类并没有放弃，而是挺身而出，用自己的智慧和勇气与敌人抗争。这种精神的力量令人感动，也让观众深思，当面临挑战时，我们应该如何坚持自己的信念，战胜困难。\n\n此外，《三体人不是无法战胜的》还向观众展现了对于不同文明的尊重与理解。在与三体人的战争中，人类不仅仅是单纯的对抗，而是在不断地探索和了解对方，从而发现对方的弱点。这种包容与理解的态度，也正是我们在现实生活中所需要的。\n\n总的来说，这部戏剧作品不仅仅是一部科幻作品，更是一部关于人类情感、信念和人性的作品。它深刻地反映了人类社会的现状，同时也向我们展示了未来的可能性。我强烈推荐观众不要错过这部精彩的戏剧作品。'}

### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果